In [11]:
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import sklearn.metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [12]:
train = pd.read_csv('dataset/data/final/train.csv').drop(columns="datasetId")
test = pd.read_csv('dataset/data/final/test.csv').drop(columns="datasetId")

In [19]:
# train
X_train = train.drop(columns='condition').to_numpy()
y_train = train['condition'].to_numpy()

# test
X_test = test.drop(columns='condition').to_numpy()
y_test = test['condition'].to_numpy()

In [21]:
# normalizing features
scaler = MinMaxScaler(feature_range=(0,1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# encode labels
one_hot_encoder = OneHotEncoder()
y_train = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot_encoder.fit_transform(y_test.reshape(-1, 1))

In [16]:
from keras.models import Sequential
from keras.layers import Dense

# Function to create model
def create_model():
    # create model
	model = Sequential()
	model.add(Dense(16, input_dim=34, activation='relu'))
	model.add(Dense(8, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model


In [23]:
model = create_model()
print('Training model...')

batch_size = 64 # 

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=150,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)

Training model...
Train on 369289 samples, validate on 41033 samples
Epoch 1/150
369289/369289 [==============================] - 16s 43us/step - loss: 0.7620 - accuracy: 0.6609 - val_loss: 0.6326 - val_accuracy: 0.7273
Epoch 2/150
369289/369289 [==============================] - 15s 42us/step - loss: 0.5638 - accuracy: 0.7650 - val_loss: 0.5264 - val_accuracy: 0.7822
Epoch 3/150
369289/369289 [==============================] - 15s 41us/step - loss: 0.4852 - accuracy: 0.8092 - val_loss: 0.4704 - val_accuracy: 0.8139
Epoch 4/150
369289/369289 [==============================] - 15s 42us/step - loss: 0.4482 - accuracy: 0.8236 - val_loss: 0.4395 - val_accuracy: 0.8232
Epoch 5/150
369289/369289 [==============================] - 16s 43us/step - loss: 0.4219 - accuracy: 0.8367 - val_loss: 0.4025 - val_accuracy: 0.8492
Epoch 6/150
369289/369289 [==============================] - 16s 42us/step - loss: 0.4000 - accuracy: 0.8478 - val_loss: 0.3946 - val_accuracy: 0.8535
Epoch 7/150
369289/369289

Epoch 108/150
369289/369289 [==============================] - 15s 41us/step - loss: 0.1668 - accuracy: 0.9394 - val_loss: 0.1805 - val_accuracy: 0.9336
Epoch 109/150
369289/369289 [==============================] - 15s 40us/step - loss: 0.1659 - accuracy: 0.9396 - val_loss: 0.1677 - val_accuracy: 0.9386
Epoch 110/150
369289/369289 [==============================] - 15s 40us/step - loss: 0.1651 - accuracy: 0.9402 - val_loss: 0.1897 - val_accuracy: 0.9289
Epoch 111/150
369289/369289 [==============================] - 15s 41us/step - loss: 0.1640 - accuracy: 0.9405 - val_loss: 0.1858 - val_accuracy: 0.9298
Epoch 112/150
369289/369289 [==============================] - 15s 40us/step - loss: 0.1641 - accuracy: 0.9403 - val_loss: 0.1854 - val_accuracy: 0.9313
Epoch 113/150
369289/369289 [==============================] - 15s 41us/step - loss: 0.1637 - accuracy: 0.9408 - val_loss: 0.1822 - val_accuracy: 0.9329
Epoch 114/150
369289/369289 [==============================] - 15s 40us/step - los

In [25]:
# save model and architecture to a single file
model.save("best_model.h5")
print("Model saved to disk")

Model saved to disk


In [ ]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model_file("best_model.h5")
model = converter.convert()

with open("best_model.tflite", "wb") as file:
    file.write(model)
    file.close()